In [14]:
import numpy as np
import torch
import math
import torch.nn as nn
import torch.nn.functional as F

In [15]:
sequence_length=4
batch_size=1
input_dim=512
d_model=512
x=torch.randn((batch_size,sequence_length,input_dim))

In [16]:
x.size()

torch.Size([1, 4, 512])

In [17]:
qkv_layer=nn.Linear(input_dim,3*d_model)

In [18]:
qkv=qkv_layer(x)

In [19]:
qkv.shape

torch.Size([1, 4, 1536])

In [20]:
num_heads=8
head_dim=d_model//num_heads
qkv=qkv.reshape(batch_size,sequence_length,num_heads,3*head_dim)

In [21]:
qkv.shape

torch.Size([1, 4, 8, 192])

In [22]:
qkv=qkv.permute(0,2,1,3)
qkv.shape

torch.Size([1, 8, 4, 192])

In [23]:
q,k,v=qkv.chunk(3,dim=-1)
q.shape,k.shape,v.shape

(torch.Size([1, 8, 4, 64]),
 torch.Size([1, 8, 4, 64]),
 torch.Size([1, 8, 4, 64]))

In [24]:
d_k=q.size()[-1]
scaled=torch.matmul(q,k.transpose(-2,-1))/math.sqrt(d_k)
scaled.shape

torch.Size([1, 8, 4, 4])

In [25]:
k.T.shape

C:\Users\Dell\AppData\Local\Temp\ipykernel_9752\3717780648.py:1: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3575.)
  k.T.shape


torch.Size([64, 4, 8, 1])

In [26]:
mask=torch.full(scaled.size(),float('-inf'))
mask=torch.triu(mask,diagonal=1)
mask[0][1]

tensor([[0., -inf, -inf, -inf],
        [0., 0., -inf, -inf],
        [0., 0., 0., -inf],
        [0., 0., 0., 0.]])

In [27]:
(scaled+mask)[0][0]

tensor([[ 0.1452,    -inf,    -inf,    -inf],
        [-0.8340, -0.3321,    -inf,    -inf],
        [ 0.0607, -0.0582, -0.1677,    -inf],
        [-0.1937, -0.0726,  0.0258,  0.4228]], grad_fn=<SelectBackward0>)

In [29]:
scaled+=mask

In [31]:
attention=F.softmax(scaled,dim=-1)

In [32]:
values=torch.matmul(attention,v)

In [40]:
import math
def scaled_dot_product(q,k,v,mask=None):
    d_k=q.size()[-1]
    scaled=torch.matmul(q,k.transpose(-1,-2))/math.sqrt(d_k)
    if mask is not None:
        scaled+=mask
    attention=F.softmax(scaled,dim=-1)
    values=torch.matmul(attention,v)
    return values,attention
    

In [41]:
values,attention = scaled_dot_product(q, k,v, mask=None)

In [42]:
attention[0][0]

tensor([[0.3379, 0.2101, 0.2154, 0.2366],
        [0.1606, 0.2654, 0.2892, 0.2848],
        [0.2300, 0.2042, 0.1830, 0.3827],
        [0.1913, 0.2160, 0.2383, 0.3544]], grad_fn=<SelectBackward0>)

In [43]:
values.size()

torch.Size([1, 8, 4, 64])

In [44]:
values=values.reshape(batch_size,sequence_length,num_heads*head_dim)
values.size()

torch.Size([1, 4, 512])

In [45]:
linear_layer=nn.Linear(d_model,d_model)

In [47]:
import math
def scaled_dot_product(q,k,v,mask=None):
    d_k=q.size()[-1]
    scaled=torch.matmul(q,k.transpose(-1,-2))/math.sqrt(d_k)
    if mask is not None:
        scaled+=mask
    attention=F.softmax(scaled,dim=-1)
    values=torch.matmul(attention,v)
    return values,attention

class MultiheadAttention(nn.Module):

    def __init__(self,imput_dim,d_model,num_heads):
        super().__init__()
        self.imput_dim=input_dim
        self.d_model=d_model
        self.num_heads=num_heads
        self.head_dim=d_model//num_heads
        self.qkv_layer=nn.Linear(input_dim,3*d_model)
        self.linear_layer=nn.Linear(d_model,d_model)


    def forward(self,x,mask=None):
        batch_size,sequence_length,input_dim=x.size()
        print(f"x.size():{x.size()}")
        qkv=self.qkv_layer(x)
        print(f"qkv.size():{qkv.size()}")
        qkv=qkv.reshape(batch_size,sequence_length,num_heads,3*head_dim)
        print(f"qkv.size():{qkv.size()}")
        qkv=qkv.permute(0,2,1,3)
        print(f"qkv.size():{qkv.size()}")
        q,k,v=qkv.chunk(3,dim=-1)
        print(f"qsize:{q.size()},ksize:{v.size()},v size:{v.size()}")
        values,attention = scaled_dot_product(q, k,v, mask=None)
        values=values.reshape(batch_size,sequence_length,num_heads*head_dim)
        out=self.linear_layer(values)
        print(f"out.size():{out.size()}")
        return out




In [49]:
input_dim=1024
d_model=512
num_heads=8

batch_size=30
sequence_length=5
x=torch.randn((batch_size,sequence_length,input_dim))
model=MultiheadAttention(input_dim,d_model,num_heads)
out=model.forward(x)

x.size():torch.Size([30, 5, 1024])
qkv.size():torch.Size([30, 5, 1536])
qkv.size():torch.Size([30, 5, 8, 192])
qkv.size():torch.Size([30, 8, 5, 192])
qsize:torch.Size([30, 8, 5, 64]),ksize:torch.Size([30, 8, 5, 64]),v size:torch.Size([30, 8, 5, 64])
out.size():torch.Size([30, 5, 512])
